# Fine-Tuning & Evaluation LaBSE with 5-fold cross validation

##### Notebook was executed in Google Colab

In [ ]:
!pip install datasets transformers seqeval -q

     |████████████████████████████████| 245kB 26.7MB/s 
     |████████████████████████████████| 2.5MB 49.9MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 245kB 51.1MB/s 
     |████████████████████████████████| 122kB 58.9MB/s 
     |████████████████████████████████| 3.3MB 51.7MB/s 
     |████████████████████████████████| 901kB 54.0MB/s 


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

# Insert utils folder into path
sys.path.insert(1, '/content/drive/MyDrive/Thesis/utils')

## Training / Fine-tuning Process

In [ ]:
task = "ner"
model_checkpoint = "sentence-transformers/LaBSE" # LaBSE pre-trained from HuggingFace Hub
batch_size = 16

### Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric, concatenate_datasets

datasets = load_dataset("conll2003")

all_data = concatenate_datasets([datasets['train'], datasets['validation'], datasets['test']])

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


In [ ]:
label_list = all_data.features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

### Processing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = all_data.map(tokenize_and_align_labels, batched=True)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
n = 5
seed = 40
kf = KFold(n_splits=n, random_state=seed, shuffle=True)

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        "LOC-f1": results['LOC']["f1"],
        "LOC-precision": results['LOC']["precision"],
        "LOC-recall": results['LOC']["recall"],
    }

In [ ]:
from cross_validation_LaBSE import cross_validation_LaBSE


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import EarlyStoppingCallback

fold = 1

args = TrainingArguments(
    output_dir=f"fold-{fold}-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True,
)

metric = load_metric("seqeval")
data_collator = DataCollatorForTokenClassification(tokenizer)

for train_index, eval_index in kf.split(tokenized_dataset):
    
    train_data = tokenized_dataset.select(train_index)
    eval_data = tokenized_dataset.select(eval_index)
    
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
    
    trainer = Trainer(model,
                      args,
                      train_dataset=train_data,
                      eval_dataset=eval_data,
                      data_collator=data_collator,
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics)
    
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
    
    trainer.train()
    
    cross_validation_LaBSE(model, tokenizer, label_list, fold)
    
    fold += 1


Some weights of BertForTokenClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, ner_tags, id, tokens, pos_tags.
***** Running training *****
  Num examples = 16595
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5190


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Loc-f1,Loc-precision,Loc-recall
1,0.062000,0.059483,0.935090,0.931061,0.933071,0.985804,0.954386,0.942065,0.967033
2,0.033600,0.051351,0.937913,0.950306,0.944069,0.988267,0.965729,0.966041,0.965417
3,0.018700,0.050580,0.945040,0.953275,0.949140,0.989319,0.967335,0.967961,0.966710
4,0.014700,0.052494,0.945887,0.955668,0.950753,0.989519,0.968264,0.970149,0.966387
5,0.009700,0.054150,0.946390,0.954998,0.950674,0.989466,0.968387,0.966516,0.970265


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, ner_tags, id, tokens, pos_tags.
***** Running Evaluation *****
  Num examples = 4149
  Batch size = 16
Saving model checkpoint to fold-1-ner/checkpoint-1038
Configuration saved in fold-1-ner/checkpoint-1038/config.json
Model weights saved in fold-1-ner/checkpoint-1038/pytorch_model.bin
tokenizer config file saved in fold-1-ner/checkpoint-1038/tokenizer_config.json
Special tokens file saved in fold-1-ner/checkpoint-1038/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, ner_tags, id, tokens, pos_tags.
***** Running Evaluation *****
  Num examples = 4149
  Batch size = 16
Saving model checkpoint to fold-1-ner/checkpoint-2076
Configuration saved in fold-1-ner/checkpoint-2076/config.json
Model weights saved in

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens.
***** Running Prediction *****
  Num examples = 1044
  Batch size = 8


Evaluation mode: strict
fp: 1833 | tp: 4007 | fn: 1536
precision: 0.686 | recall: 0.723 | f-score: 0.704 | accuracy: 0.723
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 258 | tp: 5582 | fn: 525
precision: 0.956 | recall: 0.914 | f-score: 0.934 | accuracy: 1.007
------------------------------------------------------------------------



...

Evaluation mode: strict
fp: 467 | tp: 1588 | fn: 1010
precision: 0.773 | recall: 0.611 | f-score: 0.683 | accuracy: 0.611
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 75 | tp: 1980 | fn: 860
precision: 0.964 | recall: 0.697 | f-score: 0.809 | accuracy: 0.762
------------------------------------------------------------------------



loading configuration file https://huggingface.co/sentence-transformers/LaBSE/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0dacea19c849b8846d621bf58a1aaeabc1d0d7cf63e337c0539b22e068672217.8c37167d9ff226182e8d39b551a1ee5d071e918f35157e07977ee19a100d6182
Model config BertConfig {
  "_name_or_path": "old_models/LaBSE/0_Transformer",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Loc-f1,Loc-precision,Loc-recall
1,0.062000,0.054492,0.945629,0.937417,0.941505,0.986950,0.963251,0.964152,0.962352
2,0.037000,0.053077,0.950194,0.945441,0.947812,0.988212,0.968365,0.970028,0.966708


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, ner_tags, id, tokens, pos_tags.
***** Running Evaluation *****
  Num examples = 4148
  Batch size = 16
Saving model checkpoint to fold-1-ner/checkpoint-1038
Configuration saved in fold-1-ner/checkpoint-1038/config.json
Model weights saved in fold-1-ner/checkpoint-1038/pytorch_model.bin
tokenizer config file saved in fold-1-ner/checkpoint-1038/tokenizer_config.json
Special tokens file saved in fold-1-ner/checkpoint-1038/special_tokens_map.json
Deleting older checkpoint [fold-1-ner/checkpoint-3114] due to args.save_total_limit
Deleting older checkpoint [fold-1-ner/checkpoint-5190] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, ner_tags, id, tokens, pos_tags.
***** Running Evaluation *****
  Num exam

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Loc-f1,Loc-precision,Loc-recall
1,0.062000,0.054492,0.945629,0.937417,0.941505,0.986950,0.963251,0.964152,0.962352
2,0.037000,0.053077,0.950194,0.945441,0.947812,0.988212,0.968365,0.970028,0.966708


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import EarlyStoppingCallback

fold = 1

args = TrainingArguments(
    output_dir=f"fold-{fold}-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True,
)

metric = load_metric("seqeval")
data_collator = DataCollatorForTokenClassification(tokenizer)

for train_index, eval_index in kf.split(tokenized_dataset):
    print(fold)
    if fold == 5:
      train_data = tokenized_dataset.select(train_index)
      eval_data = tokenized_dataset.select(eval_index)
      
      model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
      
      trainer = Trainer(model,
                        args,
                        train_dataset=train_data,
                        eval_dataset=eval_data,
                        data_collator=data_collator,
                        tokenizer=tokenizer,
                        compute_metrics=compute_metrics)
      
      trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
      
      trainer.train()
      
      cross_validation_LaBSE(model, tokenizer, label_list, fold)
    
    fold += 1



1
2
3
4
5


Some weights of BertForTokenClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, tokens, ner_tags, id, pos_tags.
***** Running training *****
  Num examples = 16596
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5190


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Loc-f1,Loc-precision,Loc-recall
1,0.062600,0.053931,0.945672,0.941476,0.943569,0.987123,0.966432,0.969915,0.962974
2,0.036500,0.052465,0.951497,0.948084,0.949787,0.988731,0.969669,0.969518,0.969820
3,0.018900,0.049113,0.955988,0.955446,0.955717,0.989648,0.970657,0.968702,0.972620
4,0.012600,0.052573,0.955562,0.956013,0.955787,0.989794,0.969193,0.959732,0.978843
5,0.009200,0.053280,0.959883,0.959883,0.959883,0.990525,0.972395,0.969388,0.975420


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, tokens, ner_tags, id, pos_tags.
***** Running Evaluation *****
  Num examples = 4148
  Batch size = 16
Saving model checkpoint to fold-1-ner/checkpoint-1038
Configuration saved in fold-1-ner/checkpoint-1038/config.json
Model weights saved in fold-1-ner/checkpoint-1038/pytorch_model.bin
tokenizer config file saved in fold-1-ner/checkpoint-1038/tokenizer_config.json
Special tokens file saved in fold-1-ner/checkpoint-1038/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: chunk_tags, tokens, ner_tags, id, pos_tags.
***** Running Evaluation *****
  Num examples = 4148
  Batch size = 16
Saving model checkpoint to fold-1-ner/checkpoint-2076
Configuration saved in fold-1-ner/checkpoint-2076/config.json
Model weights saved in

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens.
***** Running Prediction *****
  Num examples = 1044
  Batch size = 8


...

Evaluation mode: strict
fp: 305 | tp: 975 | fn: 343
precision: 0.762 | recall: 0.740 | f-score: 0.751 | accuracy: 0.740
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 92 | tp: 1188 | fn: 254
precision: 0.928 | recall: 0.824 | f-score: 0.873 | accuracy: 0.901
------------------------------------------------------------------------



No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens.
***** Running Prediction *****
  Num examples = 887
  Batch size = 8


Evaluation mode: strict
fp: 1699 | tp: 3359 | fn: 1729
precision: 0.664 | recall: 0.660 | f-score: 0.662 | accuracy: 0.660
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 752 | tp: 4306 | fn: 1219
precision: 0.851 | recall: 0.779 | f-score: 0.814 | accuracy: 0.846
------------------------------------------------------------------------



No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens.
***** Running Prediction *****
  Num examples = 346
  Batch size = 8


Evaluation mode: strict
fp: 498 | tp: 1605 | fn: 993
precision: 0.763 | recall: 0.618 | f-score: 0.683 | accuracy: 0.618
------------------------------------------------------------------------

Evaluation mode: forgiving
fp: 93 | tp: 2010 | fn: 835
precision: 0.956 | recall: 0.707 | f-score: 0.812 | accuracy: 0.774
------------------------------------------------------------------------

